In [2]:
!pip install torch==1.13.0 torchtext==0.14.0 torchdata==0.5.0
!pip install scipy==1.7.3 scikit-learn==1.0.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.1/890.1 MB 702.0 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 66.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 81.0 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 56.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0
    Uninstalling torch-2.0.0:
      Successfully uninstalled torch-2.0.0
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.15.1
    Uninstalling torchtext-0.15.1:
      Successfully uninstalled torchtext-0.15.1
  Attempting uninstall: torchdata
    Found existing installation: torchdata

In [6]:
import torch


In [7]:
from torch import nn

In [8]:
from torch.utils.data import DataLoader
from torchtext.transforms import ToTensor

In [9]:
from torchtext.datasets import IMDB
train_iter = iter(IMDB(split='train'))
labels = []
texts = []
for label, text in train_iter:
    labels += [label]
    texts += [text]

# определим список классов в датасете, взяв только уникальные метки,
# превратив список классов в set и затем обратно в list, и
# отсортировав их "для красоты" с помощью функции sorted
CLASSES = sorted(list(set(labels)))
print(CLASSES)

[1, 2]


 Токенайзер -- модель, разделяющая тексты на токены.

In [40]:
from torchtext.data.utils import get_tokenizer


# загрузим базовый англоязычный basic_english токенизатор с помощью get_tokenizer
tokenizer = get_tokenizer('basic_english')

 Словарь.
 Собираем словарь(проиндексированные токены для векторизации текстов). Будем строить словарь на основе итерирования по токенам по обучающей выборки.




In [41]:
from torchtext.vocab import build_vocab_from_iterator


def yield_tokens(data_iter):
    # итерируясь по данным, токенизируем текст в каждом примере в датасете
    for label, text in data_iter:
        yield tokenizer(text)


# добавим в словарь специальные токены для паддинга и для out-of-vocabulary слов
PAD_TOKEN = "<pad>"
UNK_TOKEN = "<unk>"
# выберем максимальную длину последовтаельности
MAX_LENGTH = 128
# зададим размер батча
BATCH_SIZE = 64

# а теперь воспользуемся готовой функцией `build_vocab_from_iterator`
# для того, чтобы собрать словарь токенов из представленого
# итератора yield_tokens(train_iter) по текстам обучающей выборки,
# добавим специальные токены [UNK_TOKEN, PAD_TOKEN] с помощью аргумента secials,
custom_vocab = build_vocab_from_iterator(yield_tokens(train_iter),
                                         specials=[UNK_TOKEN, PAD_TOKEN])
# причем специальном токену для out-of-vocabulary слов назначим
# специальный индекс с помощью метода set_default_index
custom_vocab.set_default_index(custom_vocab[UNK_TOKEN])

Давайте теперь сформируем пайплайны:

    обработки текстов -- токенизируем и переводим в индексы словаря
    обработки лейблов -- переводим строки neg и pos в бинарную разметку 0 и 1 соответственно.


In [42]:
# Пайплайн векторизации текста : разбиение текста на токены(последовательное применение
# токенайзера) -- и словаря -- замены токенов
# на индексы слов в словаре
# максимальную длина текста MAX_LENGTH.
# Создаем пайплайн как lambda-функцию-токенизирует входной текст с помощью tokenizer,
# затем применяет custom_vocab к токенам,
# а потом ограничивает длину листа индексированных токенов до MAX_LENGTH

vocab_tokenizer_pipeline = lambda x: custom_vocab(tokenizer(x))[:MAX_LENGTH]

In [43]:
# пайплайн векторизации меток классов представляет из себя индексирование
# меток в словаре классов.
# создадим пайплайн как lambda-функцию, которая применяет метод index
# к словарю CLASSES и преобразует результат в int

label_pipeline = lambda x: int(CLASSES.index(x))

In [44]:
label_pipeline(1),label_pipeline(2)

(0, 1)

In [45]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


Создаем DataLoader (итерируемый объект, который содержит метки класса, токенизированные и проиндексированные в соответствии со словарем тексты и длины реплик. Одна итерация возвращает нам ровно BATCH_SIZE примеров), который будет подгружать в память токенизированные и переведенные в индексы тексты и целочисленные метки.

In [46]:
def collate_batch(batch):
    """Функция collate_fn нужна DataLoader для того, чтоыб преобразовывать
    получаемый на вход лист примеров (батч в виде листа исходных данных)
    в мини-батч, представляющий из себя тензор.

    Возвращает:
    - label_list - метки классов в целочисленном виде, тензор на девайсе,
    - text_list - токенизированные и проиндексированные тексты, тензор на девайсе,
    - lengths - длины текстов в токенах, тензор на cpu
                (используется nn.utils.rnn.pack_padded_sequence)
    """
    label_list, text_list, lengths = [], [], []
    for (_label, _text) in batch:
        # добавим в список label_list преобразованный с помощью
        # label_pipeline лейбл
        label_list.append(label_pipeline(_label))
        # добавим в список text_list преобразованный с помощью
        # vocab_tokenizer_pipeline текст
        text_list.append(vocab_tokenizer_pipeline(_text))
        # добавим в список lengths длину токенизированного текста
        lengths.append(torch.tensor(len(vocab_tokenizer_pipeline(_text)),
                                    dtype=torch.int64))

    # преобразуем label_list в torch.tensor с целочисленными значениями
    label_list = torch.tensor(label_list, dtype=torch.int64)
    # преобразуем lengths в torch.tensor с целочисленными значениями
    lengths = torch.tensor(lengths, dtype=torch.int64)
    # преобразуем text_list в тензор с помощью ToTensor,
    # задав значение для padding-а custom_vocab[PAD_TOKEN],
    # чтобы тексты были все единой длины - максимальной в батче.
    # при этом помним, что в пайплайне мы уже ограничили максимальную длину
    text_list = ToTensor(padding_value=custom_vocab[PAD_TOKEN]).forward(text_list)
    return label_list.to(device), text_list.to(device), lengths


# снова создадим итератор по обучающей выборке IMDB
train_iter = IMDB(split='train')
# Создадим DataLoader на основе train_iter,
# задав BATCH_SIZE в качестве значения batch_size,
# определим shuffle в значении False,
# задав collate_batch в качестве collate_fn
dataloader = DataLoader(train_iter,
                        batch_size=BATCH_SIZE,
                        shuffle=False,
                        collate_fn=collate_batch)

In [47]:
for idx, (label, text, lengths) in enumerate(dataloader):
    # метки класса -- это целочисленные индексы в списке классов
    print(label)
    # тексты -- это целочисленные индексы токенов текста в словаре токенов
    print(text)
    # длины -- это целочисленные длины в токенах каждого примера
    # (не более `MAX_LENGTH`, так как мы ограничили максимальную длину еще
    # в пайплайне обработки текстов)
    print(lengths)

    # все длины должны быть равны размеру батча
    print(len(label))
    print(len(text))
    print(len(lengths))
    break

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([[   13,  1568,    13,  ...,    12,   203,  2182],
        [   13,   246,  1989,  ...,     4,    12,    69],
        [   51,    70,     8,  ...,     1,     1,     1],
        ...,
        [   13,   482,    13,  ...,  5337, 75760,    18],
        [   13,    97,     9,  ...,    44,    12,    64],
        [   14,    21,    17,  ...,    21,     3,     3]], device='cuda:0')
tensor([128, 128, 101, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128,
        128, 128,  92, 128,  99, 128, 128, 128, 128, 128, 121, 128, 128, 123,
        128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128,
        128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128,
        128, 128, 128,  57, 128, 128, 128, 128])
64
64
64


Нейронная сеть

Состоит из слоя перевода индексов токенов в плотные векторные представления Embedding, двунаправленной модели долгой краткосрочной памяти (Bidirectional Long Short-term Memory) и двух последовательных полносвязных слоев, размер выхода последнего равен числу классов, а размер предпоследнего представляет из себя гипер-параметр.

Слой Embedding превращает проиндексированные токены в векторные представления токенов-плотную обучаемую матрицу, которая в данном случае инициализируется с помощью случайных чисел.

In [193]:

class BiLSTMClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, hidden_dim, num_classes, pad_idx,
                 bidirectional=True, dropout=0.1):
        super().__init__()
        # слой обучаемых векторных представлений nn.Embedding
        # с заданным индексом паддинга pad_idx в качестве аргумента padding_idx,
        # с заданным размером представлений embed_dim,
        # с заданным размером словаря vocab_size и
        # без изначальной инициализации
        self.embedding = nn.Embedding(vocab_size, embed_dim,
                                      padding_idx = pad_idx)
        # слой nn.LSTM с заданным значением dropout, получающий на вход
        # вектора размерности  embed_dim и возвращающий вектора
        # скрытой размерности hidden_dim,
        # также зададим bidirectional в качестве аргумента bidirectional
        self.bilstm = nn.LSTM(embed_dim, hidden_dim,
                              bidirectional=bidirectional,
                              dropout=dropout, batch_first=True)
        
        # линейный слой nn.Linear,
        # преобразующий вектора размерности hidden_dim * 2
        # (так как используется bidirectional LSTM) в hidden_dim
        self.fc1 = nn.Linear(hidden_dim * 2, hidden_dim)
        # линейный слой nn.Linear,
        #преобразующий вектора размерности hidden_dim * 2
        # (так как используется bidirectional LSTM) в hidden_dim
        self.fc2 = nn.Linear(hidden_dim , hidden_dim)
        # линейный слой nn.Linear,
        # преобразующий вектора размерности hidden_dim
        # в размерность числа классов num_classes
        self.fc3 = nn.Linear(hidden_dim, num_classes)
        # слой nn.Dropout с заданной величиной dropout
        #  для использования - он не имеет весов, поэтому можно
        # задать один слой и использовать его в нескольких местах в сети
        self.dropout = nn.Dropout(dropout)

    def forward(self, text, text_lengths):
        # пропустим полученные на вход тексты через слой self.embedding
        embedded = self.embedding(text)
        # преобразуем векторные представления последовательностей
        # с помощью специальной функции в формат, подходящий для LSTM
        # https://pytorch.org/docs/stable/generated/torch.nn.utils.rnn.pack_padded_sequence.html?highlight=pack_padded_sequence#torch.nn.utils.rnn.pack_padded_sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(
            embedded, text_lengths, batch_first=True, enforce_sorted=False)
        
        # пропустим последовательность через нашу self.bilstm
        packed_output, (hidden, cell) = self.bilstm(packed_embedded)
        #second_bilstm, (hidden, cell) = self.bilstm(64)
        # пропустим скрытые представлния через слой self.dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]),
                                        dim = 1))
        #packed_output, (hidden, cell) = self.bilstm(hidden)
        
        # пропустим полученные скрытые представления через слой self.fc1
        output = self.fc1(hidden)
        # пропустим полученные скрытые представления через слой self.fc2
        
        output = self.fc2(output)
        # пропустим полученные скрытые представления через слой self.fc3
       #output = self.dropout(output)
        output = self.fc2(output)
        output = self.fc2(output)
        #output = self.dropout(output)
        output = self.fc3(output)
        # пропустим полученные скрытые представления через self.dropout
        #output = self.dropout(output)
        return output

In [202]:
train_iter = IMDB(split='train')


num_classes = len(CLASSES)
vocab_size = custom_vocab.__len__()

# инициализируем нашу модель BiLSTMClassificationModel,
# передав необходимые параметры:
# размер словаря,
# размер векторных представлений,
# размер скрытого пространства,
# число классов,
# индекс паддинга
# параметр, будет ли модаль двунаправиленной,
# значение дропаут,
# а также перенесем нашу модель на девайс
model = BiLSTMClassificationModel(
    vocab_size,
    embed_dim=64,
    hidden_dim=128,
    num_classes=num_classes,
    pad_idx=custom_vocab[PAD_TOKEN],
    bidirectional=True,
    dropout=0.3
).to(device)

In [144]:
import time


def train(dataloader, epoch, optimizer, criterion):
    # обязательно переводим модель в режим обучения с помощью метода train()
    model.train()
    total_acc, total_count = 0, 0
    # интервал логгирования в числе батчей
    log_interval = 100
    start_time = time.time()

    # итерируемся батчами по заданном даталоадеру
    for idx, (label, text, lengths) in enumerate(dataloader):
        # обнуляем градиенты с помощью метода zero_grad
        optimizer.zero_grad()
        # предсказываем распределение вероятностей по классам,
        # передавай в модель text и lengths
        predicted_label = model(text, lengths)
        # подсчитываем лосс с помощью criterion,
        # вычисляемого на основе predicted_label и label
        loss = criterion(predicted_label, label)
        # обратное распространение ошибки с помощью метода backward()
        loss.backward()
        # ограничиваем норму градиентов с помощью метода
        # torch.nn.utils.clip_grad_norm_, в который передаются
        # параметры модели model.parameters() и макс. значение нормы 0.1
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        # делаем шаг оптимизатора, то есть обновляем веса модели,
        # с помощью метода step()
        optimizer.step()
        # подсчитываем точность на батче
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        # подсчитываем число примеров для усреднения точности
        total_count += label.size(0)

        if idx % log_interval == 0 and idx > 0:
            # если пришло время -- логгируем
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader, criterion):
    # обязательно переводим модель в режим инференса (эвалюации)
    # с помощью метода eval()
    model.eval()
    total_acc, total_count = 0, 0

    # эвалюация не должна обновлять градиенты модели
    with torch.no_grad():
      # итерируемся батчами по заданному даталоадеру
        for idx, (label, text, lengths) in enumerate(dataloader):
            # предсказываем распределение вероятностей по классам,
            # передавай в модель text и lengths
            predicted_label = model(text, lengths)
             # подсчитываем лосс с помощью criterion,
            # вычисляемого на основе predicted_label и label
            loss = criterion(predicted_label, label)
            # предсказываем метки классов и считаем точность
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            # подсчитываем число примеров для усреднения точности
            total_count += label.size(0)
    return total_acc/total_count

In [60]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset


train_iter, test_iter = IMDB()

train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)

# определим число итоговых обучающих примеров
num_train = int(len(train_dataset) * 0.9)
# получим индексы разбиения на тренировочную и валидационную подвыборки
# с помощью функции random_split
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

# создаем DataLoader для обучающей подвыборки
train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
# создаем DataLoader для валидационной подвыборки
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
# создаем DataLoader для тестовой подвыборки
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)


In [209]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
# Hyperparameters


EPOCHS = 10 # epoch
LR = 5  # learning rate - lr

# определим функцию потерь torch.nn.CrossEntropyLoss
criterion = torch.nn.CrossEntropyLoss()
# определим оптимизатор torch.optim.SGD с заданным lr,
# передав также параметры модели model.parameters()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
# определим расписание изменения значения lr torch.optim.lr_scheduler.StepLR
# передав в качестве аргументов optimizer,
# значение step_size равное 1, и значение gamma 0.1 (коэффициент убывания)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)

total_accu = None

valid_accuracies = []

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    # запускаем обучения на 1 эпоху с помощью нашей функции train
    train(train_dataloader, epoch, optimizer, criterion)
    # подсчитываем качество на валидационной подвыборке
    # с помощью нашей функции evaluate
    accu_val = evaluate(valid_dataloader, criterion)
    valid_accuracies.append(accu_val)
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

| epoch   1 |   100/  352 batches | accuracy    0.787
| epoch   1 |   200/  352 batches | accuracy    0.787
| epoch   1 |   300/  352 batches | accuracy    0.784
-----------------------------------------------------------
| end of epoch   1 | time: 17.06s | valid accuracy    0.749 
-----------------------------------------------------------
| epoch   2 |   100/  352 batches | accuracy    0.813
| epoch   2 |   200/  352 batches | accuracy    0.806
| epoch   2 |   300/  352 batches | accuracy    0.794
-----------------------------------------------------------
| end of epoch   2 | time: 17.45s | valid accuracy    0.751 
-----------------------------------------------------------
| epoch   3 |   100/  352 batches | accuracy    0.823
| epoch   3 |   200/  352 batches | accuracy    0.827
| epoch   3 |   300/  352 batches | accuracy    0.827
-----------------------------------------------------------
| end of epoch   3 | time: 17.29s | valid accuracy    0.726 
-------------------------------

In [210]:
print('Checking the results of train dataset.')
# подсчитываем качество на обучающей подвыборке
# с помощью нашей функции evaluate
accu_train = evaluate(train_dataloader, criterion)
print('train accuracy {:8.3f}'.format(accu_train))

Checking the results of train dataset.
train accuracy    0.929


In [211]:
print('Checking the results of test dataset.')
# подсчитываем качество на тестовой подвыборке
# с помощью нашей функции evaluate
accu_test = evaluate(test_dataloader, criterion)
print('test accuracy {:8.3f}'.format(accu_test))

Checking the results of test dataset.
test accuracy    0.778
